In [80]:
import os, sys
import numpy as np
import pandas as pd
from pathlib import Path
from datetime import datetime, timedelta
from typing import List, Dict
from functools import partial
from pathlib import Path

In [81]:
output_dir = Path("capacity_data")
output_dir.mkdir(exist_ok=True, parents=True)

In [82]:
import dolphindb as ddb 
s = ddb.session()
s.connect("172.16.99.183", 38961, "", "")

True

In [83]:

q1 = 'trade_table = loadTable("dfs://tick_trade", "trades")'
q2 = 'depth_table = loadTable("dfs://tick_depth", "depths")'
s.run(q1)
s.run(q2)

In [84]:
start_date, end_date = "2022-05-21", "2022-05-24"

In [10]:
sym,hedge_sym
CROUSDC.VVS,CROUSDT.CUP
CROWBTC.VVS,CROBTC.CUP
CRODAI.VVS,CROUSDT.CUP
TONICUSDC.VVS,TONICUSDC.CUP
VVSUSDC.VVS,VVSUSDC.CUP
VVSUSDT.VVS,VVSUSDC.CUP
USDCUSDT.VVS,USDCUSDT.CUP
BIFIUSDC.VVS,BIFIUSDT.CUP
ATOMUSDC.VVS,ATOMUSDT.CUP
MMFUSDC.MMS,MMFUSDC.CUP
MMFUSDT.MMS,MMFUSDC.CUP
CROUSDC.MMS,CROUSDT.CUP
ETHUSDC.MMS,ETHUSDT.CUP
SHIBUSDC.VVS,SHIBUSDT.CUP
APEUSDC.VVS,APEUSDT.CUP
CROWBTC.MMS,CROBTC.CUP
ETHWBTC.MMS,ETHBTC.CUP
ETHCRO.MMS,ETHCRO.CUP
ETHCRO.VVS,ETHCRO.CUP
USDCUSDT.VVS,USDCUSDT.CUP
DAIUSDC.MMS,DAIUSDC.CUP



NameError: name 'sym' is not defined

In [85]:
symbols = {
    "ETHUSDC.UNS": ["ETHUSDT.CUP"],
    "MMFUSDC.MMS": ["MMFUSDC.CUP"],
    "MMFUSDT.MMS": ["MMFUSDC.CUP"],
    "CRODAI.VVS":  ["CROUSDT.CUP"],
}

crossing_pairs = set([
    "USDCUSDT", 
    "DAIUSDT",
])

coins = set([
    "ALI",
    "ATOM",
    "AVAX",
    "BIFI",
    "BTC",
    "CRO",
    "DAI",
    "ELON",
    "ETH",
    "MMF",
    "SOL",
    "TONIC",
    "USDC",
    "USDT",
    "VVS",
    "WBTC",
    "WCRO",
    "WETH",
])

In [86]:
gas_fees = {
    "UNS": 30,
    "MMS": 0.5,
    "VVS": 0.5,
}

delays = {
    "UNS": 15,
    "MMS": 6,
    "VVS": 6,
}

In [87]:
starts, ends = [], []
start = datetime.strptime(f"{start_date} 00:00:00", "%Y-%m-%d %H:%M:%S")
end = datetime.strptime(f"{end_date} 00:00:00", "%Y-%m-%d %H:%M:%S")
while start < end:
    starts.append(start.strftime("%Y.%m.%dT%H:%M:%S"))
    start += timedelta(days=1)
    ends.append(start.strftime("%Y.%m.%dT%H:%M:%S"))

In [88]:
def get_data_date(sym: str, hedge: str, side: str, delay: int, start: str, end: str, n_levels: int = 20) -> pd.DataFrame:
    t1  = "t1 = select timestamp, sum(volume) as volume, wavg(price, volume) as vwap from trade_table"
    t1 += " where timestamp>={}, timestamp<{}, symbol='{}', side={} group by timestamp order by timestamp".format(
        start, end, sym, side)

    price_type = 'a' if side == '1' else 'b'
    opposite_price_type = 'b' if side == '1' else 'a'
    volume_type = price_type + 'v'

    t2  = "t2 = select timestamp, " + price_type + f", {price_type}".join(map(str, list(range(1, n_levels + 1))))
    t2 += ', '
    t2 += volume_type + f", {volume_type}".join(map(str, list(range(1, n_levels + 1))))
    t2 += " from depth_table where timestamp>={}, timestamp<{}, symbol='{}', {}1>0 order by timestamp".format(
        start, end, hedge, price_type)

    t3  = "select * from wj(t1, t2, {}s:{}s, <[".format(delay, delay + 1)
    for level in range(1, n_levels + 1):
        t3 += f"first({price_type}{level}) as {price_type}{level}, first({volume_type}{level}) as {volume_type}{level}, "

    t3  = t3[:-2] + "]>, `timestamp)"


    ticker, sym_ext = sym.split('.')
    hedge_ticker, hedge_ext = hedge.split('.')
    cross, reverse_cross = None, None
    if not ticker == hedge_ticker:
        if ticker[:-4] == hedge_ticker[:-4] and ticker[:-4] in coins:
            if ticker[-4:] + hedge_ticker[-4:] in crossing_pairs:
                cross = ticker[-4:] + hedge_ticker[-4:]
            elif hedge_ticker[-4:] + ticker[-4:]:
                reverse_cross = hedge_ticker[-4:] + ticker[-4:]
        elif ticker[:-3] == hedge_ticker[:-3] and ticker[:-3] in coins:
            if ticker[-3:] + hedge_ticker[-3:] in crossing_pairs:
                cross = ticker[-3:] + hedge_ticker[-3:]
            elif hedge_ticker[-3:] + ticker[-3:]:
                reverse_cross = hedge_ticker[-3:] + ticker[-3:]
        elif ticker[0] == hedge_ticker[0]:
            i = 0
            for a, b in zip(ticker, hedge_ticker):
                if a == b: i += 1
            if ticker[i:] + hedge_ticker[i:] in crossing_pairs:
                cross = ticker[i:] + hedge_ticker[i:]
            elif hedge_ticker[i:] + ticker[i:]:
                reverse_cross = hedge_ticker[i:] + ticker[i:]
        else:
            print(f"Couldn't find a crossing pair for {sym} and {hedge}!")
            return pd.DataFrame()

    s.run(t1)

    if cross is None and reverse_cross is None:
        print(f"{sym} and {hedge} do not need a middle pair.")
    else:
        if cross:
            print(f"{sym} and {hedge} using crossing pair {cross}")
            s.run("cross_tbl = select timestamp, {} as price from depth_table where symbol='{}', timestamp>={}, timestamp<{};".format(
                opposite_price_type+'1', cross + '.' + hedge_ext,  start, end))
        else:
            print(f"{sym} and {hedge} using reverse crossing pair {reverse_cross}")
            s.run("cross_tbl = select timestamp, {} as price from depth_table where symbol='{}', timestamp>={}, timestamp<{};".format(
                f"1.0 / {price_type}1", reverse_cross + '.' + hedge_ext, start, end))
        s.run("t1 = wj(t1, cross_tbl, 0s:120s, <first(price) as price>, `timestamp)");

    s.run(t2)
    df = s.run(t3)
    
    if cross is None and reverse_cross is None:
        df["price"] = 1.0

    return df


def get_data(sym: str, hedge: str, side: str, delay: int, starts: List, ends: List) -> pd.DataFrame:
    n = len(starts)
    if not n > 0: return pd.DataFrame()

    res = []
    for i in range(n):
        print(f"Loading data {sym} from {starts[i]}")
        df_tmp = get_data_date(sym, hedge, side, delay, starts[i], ends[i])
        if not df_tmp.empty:
            res.append(df_tmp)

    if len(res) > 0:
        return pd.concat(res)

    return pd.DataFrame()

In [89]:
def buy_match(fee, n_levels, x):
    res, pnl = 0, 0
    tot = x["volume"]
    for i in range(1, n_levels + 1):
        price = x["vwap"] * x["price"]
        if price < x[f"a{i}"] or tot <= 0:
            break

        vol = min(tot, x[f"av{i}"])
        res += vol * x[f"a{i}"]
        pnl += vol * (price - x[f"a{i}"])
        tot -= vol

    if pnl < fee:
        return 0, 0

    pnl -= fee 

    return res, pnl

def sell_match(fee, n_levels, x):
    res, pnl = 0, 0
    tot = x["volume"]
    for i in range(1, n_levels + 1):
        price = x["vwap"] * x["price"]
        if price > x[f"b{i}"] or tot <= 0:
            break

        vol = min(tot, x[f"bv{i}"])
        res += vol * x[f"b{i}"]
        pnl += vol * (x[f"b{i}"] - price)
        tot -= vol

    if pnl < fee:
        return 0, 0

    pnl -= fee

    return res, pnl


In [78]:
results = []
for sym, hedges in symbols.items():
    _, exch = sym.split('.')
    gas = gas_fees[exch]
    delay = delays[exch]
    print(f"delay: {delay}, symbol: {sym}")
    for hedge in hedges:
        df_L = get_data(sym, hedge, '1', delay, starts, ends)
        df_L = df_L.loc[(df_L["vwap"] > 0) & (df_L["volume"] > 0) & (df_L["price"] > 0)]

        df_S = get_data(sym, hedge, '-1', delay, starts, ends)
        df_S = df_S.loc[(df_S["vwap"] > 0) & (df_S["volume"] > 0) & (df_S["price"] > 0)]

        buy_func = partial(buy_match, gas, 20)
        sell_func = partial(sell_match, gas, 20)

        buys = df_L.apply(buy_func, axis=1)
        sells = df_S.apply(sell_func, axis=1)

        df_L["potential_match"] = buys.str[0]
        df_L["potential_pnl"] = buys.str[1]
        df_S["potential_match"] = sells.str[0]
        df_S["potential_pnl"] = sells.str[1]

        df_L["timestamp"] = pd.to_datetime(df_L["timestamp"])
        df_S["timestamp"] = pd.to_datetime(df_S["timestamp"])

        df_L.set_index("timestamp", inplace=True)
        df_S.set_index("timestamp", inplace=True)

        df = pd.DataFrame({
            "LongMatch": df_L["potential_match"].resample("1D").sum(),
            "LongPnL": df_L["potential_pnl"].resample("1D").sum(),
            "ShortMatch": df_S["potential_match"].resample("1D").sum(),
            "ShortPnL": df_S["potential_pnl"].resample("1D").sum(),
        })

        df["symbol"] = sym
        df["hedge"] = hedge
        df["delay"] = delay

        results.append(df)

        output_file = output_dir / f"{sym}-{hedge}.csv"
        df.to_csv(output_file, index=False)

df = pd.concat(results)

delay: 10, symbol: ETHUSDC.UNS
Loading data ETHUSDC.UNS from 2022.05.21T00:00:00
ETHUSDC.UNS and ETHUSDT.CUP using crossing pair USDCUSDT
Loading data ETHUSDC.UNS from 2022.05.22T00:00:00
ETHUSDC.UNS and ETHUSDT.CUP using crossing pair USDCUSDT
Loading data ETHUSDC.UNS from 2022.05.23T00:00:00
ETHUSDC.UNS and ETHUSDT.CUP using crossing pair USDCUSDT
Loading data ETHUSDC.UNS from 2022.05.21T00:00:00
ETHUSDC.UNS and ETHUSDT.CUP using crossing pair USDCUSDT
Loading data ETHUSDC.UNS from 2022.05.22T00:00:00
ETHUSDC.UNS and ETHUSDT.CUP using crossing pair USDCUSDT
Loading data ETHUSDC.UNS from 2022.05.23T00:00:00
ETHUSDC.UNS and ETHUSDT.CUP using crossing pair USDCUSDT
delay: 10, symbol: MMFUSDC.MMS
Loading data MMFUSDC.MMS from 2022.05.21T00:00:00
MMFUSDC.MMS and MMFUSDC.CUP do not need a middle pair.
Loading data MMFUSDC.MMS from 2022.05.22T00:00:00
MMFUSDC.MMS and MMFUSDC.CUP do not need a middle pair.
Loading data MMFUSDC.MMS from 2022.05.23T00:00:00
MMFUSDC.MMS and MMFUSDC.CUP do not n

In [79]:
df

,LongMatch,LongPnL,ShortMatch,ShortPnL,symbol,hedge,delay
timestamp,,,,,,,
2022-05-21,5.847956e+06,3226.667097,9.292044e+06,3917.748987,ETHUSDC.UNS,ETHUSDT.CUP,10
2022-05-22,1.233484e+07,6337.215776,1.708577e+07,11373.214798,ETHUSDC.UNS,ETHUSDT.CUP,10
2022-05-21,5.148011e+05,2300.975135,5.428044e+05,3065.272554,MMFUSDC.MMS,MMFUSDC.CUP,10
2022-05-22,4.818064e+05,2002.945929,4.086013e+05,1467.576589,MMFUSDC.MMS,MMFUSDC.CUP,10
2022-05-23,7.370438e+05,4383.644293,1.007685e+06,9943.784405,MMFUSDC.MMS,MMFUSDC.CUP,10
2022-05-21,2.520749e+05,1177.288876,2.067105e+05,833.126605,MMFUSDT.MMS,MMFUSDC.CUP,10
2022-05-22,1.860099e+05,868.030793,1.371120e+05,690.628963,MMFUSDT.MMS,MMFUSDC.CUP,10
2022-05-23,2.685085e+05,1600.762888,3.992546e+05,4077.906059,MMFUSDT.MMS,MMFUSDC.CUP,10
2022-05-21,1.045892e+04,129.522601,1.715541e+03,3.609338,CRODAI.VVS,CROUSDT.CUP,10


In [71]:
df_S

,volume,vwap,price,b1,bv1,b2,bv2,b3,bv3,b4,...,b17,bv17,b18,bv18,b19,bv19,b20,bv20,potential_match,potential_pnl
timestamp,,,,,,,,,,,,,,,,,,,,,
2022-05-21 00:00:05,2998.559014,0.218923,0.998801,0.2199,2255.18,0.2198,21.28,0.2196,38.16,0.2194,...,0.2156,18572.83,0.2145,35393.07,0.2134,35393.07,0.2117,831.81,658.908168,2.742849
2022-05-21 00:00:34,425.389618,0.218867,0.998801,0.2197,21.33,0.2196,38.16,0.2194,86.87,0.2192,...,0.2156,18572.83,0.2145,35393.07,0.2134,35393.07,0.2117,831.81,0.000000,0.000000
2022-05-21 00:00:40,688.147400,0.218849,0.998801,0.2197,19.91,0.2196,38.16,0.2194,86.87,0.2192,...,0.2156,18572.83,0.2145,35393.07,0.2134,35393.07,0.2117,831.81,0.000000,0.000000
2022-05-21 00:02:02,1637.714261,0.218811,0.998801,0.2202,17.62,0.2200,38.16,0.2198,86.87,0.2197,...,0.2171,0.04,0.2167,17312.72,0.2165,3768.12,0.2156,18572.83,359.802615,1.381758
2022-05-21 00:04:36,97.789576,0.219104,0.998801,0.2207,15.10,0.2206,38.16,0.2204,119.55,0.2202,...,0.2181,2149.72,0.2178,8378.72,0.2167,17312.72,0.2165,3768.12,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-05-23 16:59:19,833.703970,0.204601,0.999001,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-05-23 16:59:54,287.723882,0.204599,0.999001,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-05-23 17:00:40,119.640656,0.204593,0.999001,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
df_L.to_csv("sample_L.csv", index=True)
df_S.to_csv("sample_S.csv", index=True)


In [ ]:
df.to_csv("ETHUSDC.UNS.arb.csv", index=True)

In [47]:
df_L.loc[df_L["potential_match"] > df_L["volume"].mul(df_L["vwap"])]

,volume,vwap,price,a1,av1,a2,av2,a3,av3,a4,...,a17,av17,a18,av18,a19,av19,a20,av20,potential_match,potential_pnl
timestamp,,,,,,,,,,,,,,,,,,,,,
2022-05-21 00:29:15,50.722258,1971.521074,1.0011,1971.71,10.14350,1971.90,0.15000,1971.91,0.00150,1971.99,...,1972.70,3.15935,1972.72,0.50000,1972.76,0.83823,1972.80,3.69846,100032.174204,47.825796
2022-05-21 01:09:59,51.148689,1955.084313,1.0011,1955.29,10.23029,1955.41,7.94835,1955.48,0.15000,1955.57,...,1956.12,1.41654,1956.13,0.00051,1956.14,0.76682,1956.17,0.11006,100029.644875,50.355125
2022-05-21 07:43:29,25.231720,1981.632631,1.0013,1982.06,0.00150,1982.12,0.15000,1982.21,2.69997,1982.30,...,1982.84,9.84109,1982.85,0.75649,1982.90,2.17123,1982.93,0.03384,50020.356024,14.643976
2022-05-21 08:16:02,29.878153,1979.821090,1.0012,1980.42,10.10062,1980.49,0.15000,1980.55,0.50502,1980.59,...,1981.04,1.40093,1981.05,2.69997,1981.07,60.01799,1981.10,0.12904,59176.214585,18.166461
2022-05-21 08:33:59,82.722379,1979.873134,1.0013,1980.87,9.09660,1981.27,1.98709,1981.28,1.98709,1981.30,...,1981.65,1.37196,1981.70,9.84109,1981.73,61.73713,1981.74,3.45688,163913.206726,49.522150
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-05-22 21:00:46,16.637533,2006.976487,1.0011,2007.12,9.97222,2007.20,1.10508,2007.21,1.98709,2007.43,...,2007.82,3.24799,2007.83,1.15283,2007.87,4.63654,2007.88,2.71523,33394.899621,2.967623
2022-05-22 21:28:52,24.625386,2011.706494,1.0011,2012.11,0.01494,2012.35,3.28139,2012.36,0.15000,2012.38,...,2012.73,0.74645,2012.74,7.94835,2012.78,1.20587,2012.79,0.50000,49559.186684,4.354808
2022-05-22 21:47:57,32.000000,2014.688840,1.0011,2014.56,9.92773,2014.70,0.01500,2014.99,1.98709,2015.05,...,2015.39,1.68809,2015.40,4.46713,2015.47,0.01002,2015.49,0.12703,64478.914457,32.045482


In [ ]:

df_S.loc[df_S["potential_match"] > df_S["volume"].mul(df_S["vwap"])]

,volume,vwap,b1,bv1,b2,bv2,b3,bv3,b4,bv4,...,b17,bv17,b18,bv18,b19,bv19,b20,bv20,potential_match,potential_pnl
timestamp,,,,,,,,,,,,,,,,,,,,,
